# Introduction

This Jupyter notebook shows and run the web scrapping part of the processing of the raw data.
The other part is called from python scripts (utils and preprocessing) so it don't goes to long about DF manipulations.

## Imports, libraries & reading raw data files

In [ ]:
### IMPORTING FUNCTIONS FROM SCRIPTS ###
from preprocessing import game_data_pp_1, game_data_pp_2, game_data_pp_3, nfl_elo_pp_1, nfl_elo_pp_2, nfl_pp
from utils import to_pickle, from_pickle, search_teams_1, search_teams_2

### LIBRARIES
import pandas as pd
from tqdm import tqdm

import requests
import time
import os
import sys
import pickle

# import multiprocessing as mp

In [ ]:
game_data = pd.read_csv('raw_data/game_data.csv', index_col=0)
nfl_elo = pd.read_csv('raw_data/nfl_elo.csv')
nfl = pd.read_excel('raw_data/nfl.xlsx')

## GAME_DATA FILE

In [ ]:
display(game_data)
print(list(game_data.columns))
type(game_data['date'][0])

In [ ]:
game_data = game_data_pp_1(game_data)

display(game_data)
print(list(game_data.columns))
type(game_data['date'][0])

In [ ]:
if not os.path.isfile('pickles/TEAMS.pkl'):

    teams = {}
    links = game_data['link']

    ### Looping on game_date links, extracting abbreviations from the link and scrapping the full name on the web page.
    ### Every time it founds a new abbreviation, a new key/value pair is generated and add to the teams dictionary.
    for i, link in tqdm(enumerate(links), total=len(links)):
        URL = link
        pointer = 'teams' # Word just before 3 letter team abbreviation in this link format
        j = link.find(pointer)
        team = link[j+len(pointer)+1 : j+len(pointer)+4].upper() # Extracting abbreviation

        if i > 0 and team == list(teams)[-1]: # Evoid to repeat the process if the team is already added to the dict
            continue
        else:
            r = requests.get(URL)
            parsed = r.text
            loc = parsed.find('meta name="Description" content')

            if loc == -1: # If IP is banned from the server, parsed will not have the needed content: use proxy
                print('Banned IP: use proxy or wait before trying again')
                os.remove('pickles/TEAMS.pkl')
                break
            
            loc = loc + 52
            full_name = parsed[loc:loc+50].split(',', 1)[0]
            full_name = full_name[:-7]

            teams[team] = full_name # Adding new key/value pair to dict

        time.sleep(3) # Sleep to evoid being banned

    os.makedirs('pickles', exist_ok=True)
    to_pickle(teams, filename='pickles/TEAMS.pkl')

    del links

In [ ]:
teams = from_pickle('pickles/TEAMS.pkl')

In [ ]:
game_data = game_data_pp_2(game_data, teams)

display(game_data)
print(list(game_data.columns))

In [ ]:
if not os.path.isfile('pickles/AWAY.pkl'):

    game_data_parsing = game_data[['Season', 'Team', 'Link']]

    away = []
    for i, row in tqdm(game_data_parsing.iterrows(), total = game_data_parsing.shape[0]):
        link = row['Link']

        if (i > 0) and (link == link_last):
            continue
        else:
            URL = link + '#games'
            season = row['Season']
            team = row['Team']
            table_link = requests.get(URL).content
            df_list = pd.read_html(table_link)
            df_bis = df_list[1]
            df_bis.columns = df_bis.columns.droplevel()
            df_bis = df_bis[['Date', 'Unnamed: 8_level_1']]
            try:
                cut_playoffs = df_bis.index[df_bis['Date'] == 'Playoffs'].to_list()[0]
                df_bis = df_bis.iloc[:cut_playoffs, :]
            except:
                pass
            away_set = list(df_bis['Unnamed: 8_level_1'].values)

            gd_temp = game_data[game_data['Team'] == team]
            gd_temp = gd_temp[gd_temp['Season'] == season]

            if len(away_set) != gd_temp.shape[0]:
                raise ValueError(f'Lenghts are not matching for {team} {season} ({link}): {len(away_set)} vs {gd_temp.shape[0]}')

            link_last = link
            away += away_set

        time.sleep(3)

        os.makedirs('pickles', exist_ok=True)
        to_pickle(away, 'pickles/AWAY.pkl')

else:
    away = from_pickle('pickles/AWAY.pkl')

game_data.insert(3, 'Away?', away)

del away

In [ ]:
game_data = game_data_pp_3(game_data)

display(game_data)
print(list(game_data.columns))

## NFL_ELO FILE

In [ ]:
display(nfl_elo)
print(list(nfl_elo.columns))
type(nfl_elo['date'][0])

In [ ]:
nfl_elo, teams2add, nfl_elo_sub = nfl_elo_pp_1(nfl_elo, sub_num=10)

display(nfl_elo)
print(type(nfl_elo['date'][0]))

display(nfl_elo_sub)

temp = teams2add - set(list(teams.keys()))
print(f'The are {len(temp)} abbreviations without full names')
del temp

In [ ]:
if not os.path.isfile('pickles/TEAMS_final.pkl'):

    nfl_elo_parsing = nfl_elo_sub[['date', 'season', 'team1', 'team2']]

    for _, row in tqdm(nfl_elo_parsing.iterrows(), total = nfl_elo_parsing.shape[0]):
        if len(temp) == 0:
            break
        
        date = row['date']
        season = row['season']
        team1_i = row['team1']
        team2_i = row['team2']
        
        if (team1_i and team2_i) in teams:
            continue

        if team1_i not in teams:
            search_teams_1(season, team1_i, teams)

        if team2_i not in teams:
            search_teams_1(season, team2_i, teams)

        if (team1_i or team2_i) not in teams:
            search_teams_2(date, team1_i, team2_i, teams)

    os.makedirs('pickles', exist_ok=True)
    to_pickle(teams, 'pickles/TEAMS_final.pkl')

    del nfl_elo_parsing

else:
    teams = from_pickle('pickles/TEAMS_final.pkl')

temp = teams2add - set(list(teams.keys()))
print(f'{len(temp)} team names (full and abbreviated) still missing')

In [ ]:
nfl_elo = nfl_elo_pp_2(nfl_elo, teams)

display(nfl_elo)
print(list(nfl_elo.columns))

temp = teams2add - set(list(teams.keys()))
print(f'The are {len(temp)} abbreviations without full names')
del temp

## NFL FILE

In [ ]:
display(nfl)
print(list(nfl.columns))
type(nfl['Date'][0])

In [ ]:
nfl = nfl_pp(nfl, teams)

display(nfl)
print(list(nfl.columns))

## MERGING

In [ ]:
print(list(game_data.columns))
print(list(nfl.columns))

In [ ]:
merge_cols = ['Date', 'Home Team', 'Away Team']
merge = pd.merge(game_data, nfl, how='outer', on=merge_cols)

In [ ]:
merge